In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime
import numpy as np


In [9]:
torneo = "liga-dimayor-finalizacion"
fase = "todos contra todos"

In [10]:
partidos = pd.DataFrame(columns=['year', 'torneo', 'fase', 'jornada', 'local', 'goles_local', 'goles_visitante', 'visitante', 'fecha', 'dia', 'arbitro', 'pos_local', 'pos_visitante', 'gana_local', 'empate', 'gana_visitante'])


In [11]:

def scrap_page(url, year, jornada):

    # Create a WebDriver instance for Chrome
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    driver.get(url)
    div_element = driver.find_element(By.CSS_SELECTOR, 'div.large-8.columns')
    boxes = div_element.find_elements(By.XPATH, 'div[@class="box"] [@style="border-top: 0 !important;"]')
    
    for i in range(len(boxes)):
        l = [year, torneo, fase, jornada]
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[1]/a''').get_attribute("innerText")) # home
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[5]/span/a/span''').get_attribute("innerText").split(":")[0]) # goals home
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[5]/span/a/span''').get_attribute("innerText").split(":")[1]) # goals visitor
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[8]/a''').get_attribute("innerText")) # visitor
        l.append(datetime.strptime(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[2]/td/a''').get_attribute("href").split("/")[-1], "%Y-%m-%d").date()) # date
        l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[2]/td''').get_attribute("innerText").split(",")[0]) # day
        try:
            l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[3]/td''').get_attribute("innerText").split("·")[-1].strip()) # referee
        except NoSuchElementException:
            l.append(np.nan) 
        try:
            l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[1]/span''').get_attribute("innerText").replace("(","").replace(".)","")) # pos home
        except NoSuchElementException:
            l.append(np.nan) 
        try:
            l.append(boxes[i].find_element(By.XPATH, f'''./table/tbody/tr[1]/td[8]/span''').get_attribute("innerText").replace("(","").replace(".)","")) # pos visitor
        except NoSuchElementException:
            l.append(np.nan)     
        if ((l[5] == "-") or (l[6] == "-")):
            l=l+[np.nan,np.nan,np.nan]
        elif l[5] > l[6]:
            l=l+[True,False,False]
        elif l[5] == l[6]:
            l=l+[False,True,False]
        else:
            l=l+[False,False,True]
        partidos.loc[len(partidos)] = l
    
    driver.quit()
    return partidos

In [12]:
def get_cantidad(url):
    # Create a WebDriver instance for Chrome
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    # Visit the website
    driver.get(url)

    # Click on accept cookies policy
    # time.sleep(2)
    # WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "sp_message_iframe_851961")))
    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Aceptar']"))).click()
    # time.sleep(2)

    driver.get(url)
    selector = driver.find_elements(By.CSS_SELECTOR, 'ul.chzn-results')[1]
    fechas = selector.find_elements(By.XPATH, './*')
    cantidad = len(fechas)
    driver.quit()
    return cantidad

In [13]:
for i in range(2024,2025):
    for j in range(1,10):
        # website = f"https://www.transfermarkt.es/liga-dimayor-finalizacion/spieltag/wettbewerb/COL1/spieltag/{str(j)}/saison_id/{i-1}"
        website = f"https://www.transfermarkt.es/liga-dimayor-apertura/spieltag/wettbewerb/COLP/spieltag/{str(j)}/saison_id/{i-1}"
        print(website)
        partidos.append(scrap_page(website, i, j))
partidos["goles_local"] = partidos["goles_local"].replace("-",np.nan).astype(float)
partidos["goles_visitante"] = partidos["goles_visitante"].replace("-",np.nan).astype(float)
partidos["gana_local"] = partidos["gana_local"].astype(bool)
partidos["empate"] = partidos["empate"].astype(bool)
partidos["gana_visitante"] = partidos["gana_visitante"].astype(bool)
partidos 

https://www.transfermarkt.es/liga-dimayor-apertura/spieltag/wettbewerb/COLP/spieltag/1/saison_id/2023
10


/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_2492/1747475515.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  partidos.append(scrap_page(website, i, j))


https://www.transfermarkt.es/liga-dimayor-apertura/spieltag/wettbewerb/COLP/spieltag/2/saison_id/2023
10


/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_2492/1747475515.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  partidos.append(scrap_page(website, i, j))


https://www.transfermarkt.es/liga-dimayor-apertura/spieltag/wettbewerb/COLP/spieltag/3/saison_id/2023
10


...

https://www.transfermarkt.es/liga-dimayor-apertura/spieltag/wettbewerb/COLP/spieltag/8/saison_id/2023
10


/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_2492/1867192425.py:39: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  partidos.loc[len(partidos)] = l
/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_2492/1747475515.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  partidos.append(scrap_page(website, i, j))


https://www.transfermarkt.es/liga-dimayor-apertura/spieltag/wettbewerb/COLP/spieltag/9/saison_id/2023
10


/var/folders/g9/q6tzypkx56110bt4s_wsr1y40000gn/T/ipykernel_2492/1747475515.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  partidos.append(scrap_page(website, i, j))


,year,torneo,fase,jornada,local,goles_local,goles_visitante,visitante,fecha,dia,arbitro,pos_local,pos_visitante,gana_local,empate,gana_visitante
0,2024,liga-dimayor-finalizacion,todos contra todos,1,Patriotas,0.0,1.0,Jaguares FC,2024-01-20,sábado,Jonathan Ortíz,19,9,False,False,True
1,2024,liga-dimayor-finalizacion,todos contra todos,1,Pereira,2.0,2.0,Deportivo Cali,2024-01-20,sábado,Jorge Duarte,3,5,False,True,False
2,2024,liga-dimayor-finalizacion,todos contra todos,1,Once Caldas,2.0,1.0,Boyacá Chicó,2024-01-20,sábado,Wander Mosquera,12,18,True,False,False
3,2024,liga-dimayor-finalizacion,todos contra todos,1,Deportes Tolima,1.0,2.0,Fortaleza CEIF,2024-01-21,domingo,Ferney Trujillo,1,10,False,False,True
4,2024,liga-dimayor-finalizacion,todos contra todos,1,CD América,0.0,1.0,Águilas Doradas,2024-01-21,domingo,Nicolás Gallo,16,7,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024,liga-dimayor-finalizacion,todos contra todos,9,Envigado,1.0,0.0,Patriotas,2024-02-27,martes,Diego Ulloa,12,19,True,False,False
86,2024,liga-dimayor-finalizacion,todos contra todos,9,Millonarios,0.0,2.0,Once Caldas,2024-02-28,miércoles,Wilmar Roldán,10,13,False,False,True
87,2024,liga-dimayor-finalizacion,todos contra todos,9,Fortaleza CEIF,1.0,2.0,Alianza FC,2024-02-28,miércoles,Bismark Santiago,9,16,False,False,True
88,2024,liga-dimayor-finalizacion,todos contra todos,9,Indep. Medellín,1.0,0.0,Deportivo Pasto,2024-03-01,viernes,Nicolás Gallo,18,20,True,False,False


In [14]:
partidos.to_csv("historic_scores.csv", mode='a', index=False, header=False)